In [1]:
import requests
import pandas as pd
import json
import psycopg2
from sqlalchemy import create_engine


In [2]:
data = 'https://api.covidtracking.com/v1/states/daily.json'
d =requests.get(data)
df = pd.read_json(d.text)


In [18]:
engine = create_engine('postgresql+psycopg2://username:secret@db:5432/database')
conn = psycopg2.connect(database="database",user="username", password="secret",host="db", port="5432")
cur = conn.cursor()

In [20]:
cur.execute("""SELECT * FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cur.fetchall():
    print(table)

('database', 'public', 'proba', 'BASE TABLE', None, None, None, None, None, 'YES', 'NO', None)


In [5]:
df.head().to_sql('proba', engine, if_exists='replace',index=False)

In [6]:
cursor.execute("""SELECT proba FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)

NameError: name 'cursor' is not defined